# Iteration 1

In [1]:
const { Client } = require('@notionhq/client');


async function retrievePage(pageId) {
let notionApiKey = process.env.notion_secret;
let notion = new Client({ auth: notionApiKey });
  try {
    const response = await notion.pages.retrieve({ page_id: pageId });
    return response;
  } catch (error) {
    // Handle errors if needed
    console.error(error);
    throw error;
  }
}

var response = retrievePage('4724abef-8c3e-4736-b2b3-33680bd83929')
console.log(response)

Promise { <pending> }


In [2]:
console.log(response)

Promise {
  {
    object: 'page',
    id: '4724abef-8c3e-4736-b2b3-33680bd83929',
    created_time: '2023-11-22T19:09:00.000Z',
    last_edited_time: '2023-12-08T16:03:00.000Z',
    created_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    last_edited_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    cover: null,
    icon: { type: 'external', external: [Object] },
    parent: {
      type: 'database_id',
      database_id: 'b6b0367a-e451-48d3-b985-3464e43c6363'
    },
    archived: false,
    properties: {
      'Project (parent task)': [Object],
      Notes: [Object],
      'total subtask time': [Object],
      'when initiated': [Object],
      'work log': [Object],
      'initiation notes': [Object],
      'Last edited time': [Object],
      'total time': [Object],
      'Roadmap Item': [Object],
      'Achieve by': [Object],
      'Subtasks completion': [Object],
      URL: [Object],
      'Notes from Ginkgo log': [Object],
      'summariz

## 1.1

In [1]:
var {getCurrentTimestamp, loadJsonFile} = require('../src/fileFunctions')
const fs = require('fs');

async function retrievePage(
  pageId, jsonFileName='../data/notion_page', save = true, appendTimestamp = true
  ) {
  const { Client } = require('@notionhq/client');
  const fs = require('fs');
  let notionApiKey = process.env.notion_secret;
  let notion = new Client({ auth: notionApiKey });
    try {
      const response = await notion.pages.retrieve({ page_id: pageId });
      if (save) {
        if (appendTimestamp) {
          const timestamp = getCurrentTimestamp();
          jsonFileName = `${jsonFileName}_${timestamp}`
        }
        await fs.promises.writeFile(`${jsonFileName}.json`, JSON.stringify(response, null, 2));
        console.log(`Saved response to ${jsonFileName}.json`);
      }
      return response;
    } catch (error) {
      // Handle errors if needed
      console.error(error);
      throw error;
    }
}

var response = retrievePage('4724abef-8c3e-4736-b2b3-33680bd83929');


Saved response to ../data/notion_page_2023-12-25_1218.json


## 1.2

In [4]:


async function retrievePage(
  pageId, jsonFileName='../data/notion_page', save = true, appendTimestamp = true
) {
  const { Client } = require('@notionhq/client');
  const fs = require('fs');
  let notionApiKey = process.env.notion_secret;
  let notion = new Client({ auth: notionApiKey });
    try {
      const response = await notion.pages.retrieve({ page_id: pageId });
      if (save) {
        await saveResponseJson(response, jsonFileName, appendTimestamp);
      }
      return response;
    } catch (error) {
      console.error(error);
      throw error;
    }
}

async function saveResponseJson(data, jsonFileName, appendTimestamp) {
    var {getCurrentTimestamp} = require('../src/fileFunctions');
    const fs = require('fs');
    try {
        if (appendTimestamp) {
        const timestamp = getCurrentTimestamp();
        jsonFileName = `${jsonFileName}_${timestamp}`
        }
        await fs.promises.writeFile(`${jsonFileName}.json`, JSON.stringify(data, null, 2));
        console.log(`Saved response to ${jsonFileName}.json`);
    } catch (error) {
        console.error(error);
        throw error;
    }
}

var response = retrievePage('e40230f2-e2c3-463d-9dcf-7c3cbc88227c');

Saved response to ../data/notion_page_2023-12-25_1229.json


# Access the page response

In [14]:
response

Promise {
  {
    object: 'page',
    id: 'e40230f2-e2c3-463d-9dcf-7c3cbc88227c',
    created_time: '2023-11-22T07:08:00.000Z',
    last_edited_time: '2023-12-04T01:16:00.000Z',
    created_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    last_edited_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    cover: null,
    icon: { type: 'emoji', emoji: '☀️' },
    parent: {
      type: 'database_id',
      database_id: 'b6b0367a-e451-48d3-b985-3464e43c6363'
    },
    archived: false,
    properties: {
      'Project (parent task)': [Object],
      Notes: [Object],
      'total subtask time': [Object],
      'when initiated': [Object],
      'work log': [Object],
      'initiation notes': [Object],
      'Last edited time': [Object],
      'total time': [Object],
      'Roadmap Item': [Object],
      'Achieve by': [Object],
      'Subtasks completion': [Object],
      URL: [Object],
      'Notes from Ginkgo log': [Object],
      'summarized time tr

## **iteration 1

In [83]:
async function parsePage(response) {
    try {
        const data = await response;
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


var title = parsePage(response)
console.log(title)

Promise { <pending> }


In [84]:
console.log(title)

Promise {
  {
    Name: 'Solarathon: thinking about implementation',
    'Project tags': [ 'portfolio' ],
    Project: '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
    'Parent-task': null,
    'Sub-tasks': null
  }
}


# Parse initial time tracking data

In [33]:
var {loadJsonFile} = require('../src/fileFunctions')

var data = loadJsonFile('../data/notion_time_tracking_3m_2023-12-21_2334.json')
data

[
  {
    object: 'page',
    id: 'a7f579a6-5439-43b9-9e5d-f695889ac88f',
    created_time: '2023-12-01T04:26:00.000Z',
    last_edited_time: '2023-12-01T08:00:00.000Z',
    created_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    last_edited_by: { object: 'user', id: 'f4b38e73-d5f6-43c6-99b5-a5cbf1fb60c2' },
    cover: null,
    icon: null,
    parent: {
      type: 'database_id',
      database_id: 'bcd0d296-955b-4b92-8621-e4b96ebad29c'
    },
    archived: false,
    properties: {
      'Task Tag': [Object],
      Flag: [Object],
      'Created time': [Object],
      Notes: [Object],
      Elapsed: [Object],
      Projects: [Object],
      'Start min': [Object],
      summary: [Object],
      'End min': [Object],
      'follow up task': [Object],
      URL: [Object],
      'End hr': [Object],
      Tasks: [Object],
      'Project tag': [Object],
      'start time': [Object],
      'Project (Rollup)': [Object],
      'Start hr': [Object],
      'end time': [Obj

## Explore data

In [14]:
console.log(data.length)
console.log(Object.keys(data[0]['properties']))

100
[
  'Task Tag',     'Flag',
  'Created time', 'Notes',
  'Elapsed',      'Projects',
  'Start min',    'summary',
  'End min',      'follow up task',
  'URL',          'End hr',
  'Tasks',        'Project tag',
  'start time',   'Project (Rollup)',
  'Start hr',     'end time',
  'Roadmap Item', 'Name'
]


## Iteration 1

In [117]:


function parseTimeTracking(data) {
    const parsed_data = {};
    const relations_list = ['Tasks'];
    for (let i = 0; i < data.length; i++) {
        const item = data[i];
        const id = item['id']
        record = {};
        // console.log(item['properties'])
        for (let j = 0; j < relations_list.length; j++) {
            const relation = relations_list[j];
            // console.log(relation, item['properties'][relation])
            //  Only parse relation if it is greater than 0
            if (item['properties'][relation].length === 0) {
                record[relation] = null;
            } else {
                record[relation] = item['properties'][relation]['relation'].map(item => item['id']);
            };
        parsed_data[id] = record
        }
    }
    return parsed_data;
}


var parsed_data = parseTimeTracking(data)
console.log(parsed_data)

{
  'a7f579a6-5439-43b9-9e5d-f695889ac88f': { Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ] },
  '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': { Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ] },
  'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': { Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ] },
  '6e291503-5d5c-42ea-a86b-7813c4164b05': { Tasks: [ '196d64f7-6dc4-4b46-8178-2679482d803e' ] },
  '1b4cc854-fde5-4e4b-8e1d-d1c57bb3fc8f': { Tasks: [ '1e819f7b-da96-45ac-9470-b0c108c3c9d2' ] },
  'b4107914-0a39-49c7-b639-61c8facd801c': { Tasks: [ '196d64f7-6dc4-4b46-8178-2679482d803e' ] },
  'd03e6f6f-41d8-41ae-8a39-1aa2d298c81d': { Tasks: [ 'a6405712-c50c-4559-8d1f-e763a5cf7dae' ] },
  'bda498a5-e6f6-47dd-8ada-b725669576f1': { Tasks: [ 'afc41cac-f014-441c-a13b-223a0a2e482b' ] },
  '00a94768-dae6-4b70-989d-dd8f0238eeed': { Tasks: [ '1020f71e-cc88-44b5-847c-711b9f28f68a' ] },
  '148db728-8d0d-46fa-995f-8a1f00180c61': { Tasks: [ '330197b7-8921-4f6f-81b2-a7c942df647c' ] },
  '8c137311-d17d-45c0-baf5-6

In [113]:
console.log(parsed_data)

{ Tasks: 'a2700a95-3881-40a7-921f-732e8a8617a0' }


## 1.1

In [216]:


function parseTimeTracking(data) {
    const parsed_data = {};
    const relations_list = ['Tasks'];
    const array_types = ['multi_select', 'relation']
    let properties =  Object.keys(data[0]['properties'])
    const to_ignore = [
            'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 
            'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'
        ]
    properties = properties.filter(item => !to_ignore.includes(item))
    for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
        const item = data[i];
        const id = item['id']
        record = {};
        for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            // console.log(property_dict)
            const property_type = property_dict['type'];
            // console.log(property_type)
            if (property_type == 'relation') {
                //  Only parse relation if it is greater than 0
                if (property_dict[property_type].length === 0) {
                    record[property] = null;
                } else {
                    record[property] = property_dict[property_type].map(item => item['id']);
                };
            } else if (property_type == 'rollup') {
                rollup_type = property_dict[property_type]['type'];
                if (rollup_type == 'array' && property_dict[property_type]['array'].length > 0) {
                    var array_type = property_dict[property_type]['array'][0]['type'];
                    if (array_type == 'multi_select' || array_type == 'relation') {
                        record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                    } else {
                        record[property] = null
                    };
                }
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'formula') {
                formula_type = property_dict[property_type]['type'];
                record[property] = property_dict[property_type][formula_type];
            } else if (property_type == 'multi_select') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type].map(item => item);
                } else {
                    record[property] = null;
                }
            } else {
                console.log(`Property of type ${property_type} was not parsed: ${property}`);
            }
            
        parsed_data[id] = record
        }
    }
    return parsed_data;
}


var parsed_data = parseTimeTracking(data)
console.log(parsed_data)

{
  'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 3.566666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '20:26',
    'end time': '24:00',
    'Roadmap Item': []
  },
  '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.2,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '16:03',
    'end time': '17:15',
    'Roadmap Item': []
  },
  'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.166666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '14:53',
    'end time': '16:03',
    'Roadmap Item': []
  }
}


## 1.2 Save the response as JSON

In [218]:


function parseTimeTracking(
    data, jsonFileName='../data/notion_time_tracking_parsed', save = true, appendTimestamp = true
    ) {
    const parsed_data = {};
    const relations_list = ['Tasks'];
    const array_types = ['multi_select', 'relation']
    let properties =  Object.keys(data[0]['properties'])
    const to_ignore = [
            'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 
            'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'
        ]
    properties = properties.filter(item => !to_ignore.includes(item))
    for (let i = 0; i < data.length; i++) {
        const item = data[i];
        const id = item['id']
        record = {};
        for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            // console.log(property_dict)
            const property_type = property_dict['type'];
            // console.log(property_type)
            if (property_type == 'relation') {
                //  Only parse relation if it is greater than 0
                if (property_dict[property_type].length === 0) {
                    record[property] = null;
                } else {
                    record[property] = property_dict[property_type].map(item => item['id']);
                };
            } else if (property_type == 'rollup') {
                rollup_type = property_dict[property_type]['type'];
                if (rollup_type == 'array' && property_dict[property_type]['array'].length > 0) {
                    var array_type = property_dict[property_type]['array'][0]['type'];
                    if (array_type == 'multi_select' || array_type == 'relation') {
                        record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                    } else {
                        record[property] = null
                    };
                }
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'formula') {
                formula_type = property_dict[property_type]['type'];
                record[property] = property_dict[property_type][formula_type];
            } else if (property_type == 'multi_select') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type].map(item => item);
                } else {
                    record[property] = null;
                }
            } else {
                console.log(`Property of type ${property_type} was not parsed: ${property}`);
            }
            
        parsed_data[id] = record
        }
    }
    if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
    };
    return parsed_data;
}


var parsed_data = parseTimeTracking(data)
console.log(parsed_data)

{
  'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 3.566666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '20:26',
    'end time': '24:00',
    'Roadmap Item': []
  },
  '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.2,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '16:03',
    'end time': '17:15',
    'Roadmap Item': []
  },
  'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.166666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'start time': '14:53',
    'end time': '16:03',
    'Roadmap Item': []
  },
  '6e291503-5d5c-42ea-a86b-7813c4164b05': {
    'Task Tag': [ 'Ginkgo' ],
    Flag: null,
    Notes: '2023-11-30 12:25 Publish exercise 541, first exercise for TMA to try fixing B-A loop issue.\n' +
   

# Parse time tracking data and add task data

In [2]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId) {
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


function parseTimeTracking(
    data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
    ) {
    const parsed_data = {};
    const relations_list = ['Tasks'];
    const array_types = ['multi_select', 'relation']
    let properties =  Object.keys(data[0]['properties'])
    const to_ignore = [
            'Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 
            'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'
        ]
    properties = properties.filter(item => !to_ignore.includes(item))
    for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
        const item = data[i];
        const id = item['id']
        record = {};
        for (let j = 0; j < properties.length; j++) {
            const property = properties[j];
            const property_dict = data[i]['properties'][property];
            // console.log(property_dict)
            const property_type = property_dict['type'];
            // console.log(property)
            if (property_type == 'relation') {
                //  Only parse relation if it is greater than 0
                if (property_dict[property_type].length === 0) {
                    record[property] = null;
                } else {
                    record[property] = property_dict[property_type].map(item => item['id']);
                    if (property == 'Tasks') {
                        console.log('Parsing task details');
                        // record['Task details'] = property_dict[property_type].map(item => parsePage(item['id']));
                        //  Convert Promise object to string
                        record['Task details'] = property_dict[property_type].map(async item => await parsePage(item['id']));
                        
                    }
                };
            } else if (property_type == 'rollup') {
                rollup_type = property_dict[property_type]['type'];
                if (rollup_type == 'array' && property_dict[property_type]['array'].length > 0) {
                    var array_type = property_dict[property_type]['array'][0]['type'];
                    if (array_type == 'multi_select' || array_type == 'relation') {
                        record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
                    } else {
                        record[property] = null
                    };
                } else {
                    record[property] = null
                }
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'rich_text') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type][0]['text']['content'];
                } else {
                    record[property] = null;
                };
            } else if (property_type == 'formula') {
                formula_type = property_dict[property_type]['type'];
                record[property] = property_dict[property_type][formula_type];
            } else if (property_type == 'multi_select') {
                if (property_dict[property_type].length > 0) {
                    record[property] = property_dict[property_type].map(item => item);
                } else {
                    record[property] = null;
                }
            } else {
                console.log(`Property of type ${property_type} was not parsed: ${property}`);
            }
            
        parsed_data[id] = record
        }
    }
    if (save) {
        saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
    };
    return parsed_data;
}


var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

Parsing task details
Parsing task details
Parsing task details
{
  'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 3.566666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'Task details': [ [Promise] ],
    'start time': '20:26',
    'end time': '24:00',
    'Roadmap Item': []
  },
  '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.2,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'Task details': [ [Promise] ],
    'start time': '16:03',
    'end time': '17:15',
    'Roadmap Item': []
  },
  'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
    'Task Tag': [ 'life' ],
    Flag: null,
    Notes: null,
    Elapsed: 1.166666666667,
    Tasks: [ '29889961-8494-4d87-8458-567104699cc2' ],
    'Task details': [ [Promise] ],
    'start time': '14:53',
    'end time': '16:03',
    'Roadmap Item': []
  }
}


Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1621.json


## Iteration 2 with Codeium's help 

In [3]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId) {
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          record[property] = property_dict[property_type].map(item => item['id']);

          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = property_dict[property_type].map(item => parsePage(item['id']));
            record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}


var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1631.json


In [4]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '14:53',
      'end time': '16:03',
      'Roadmap Item': []
    }
  }
}


## 2.1

In [13]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId) {
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
          const relationIds = [];
      
          for (let k = 0; k < relationValues.length; k++) {
            relationIds.push(relationValues[k]['id']);
          }
      
          record[property] = relationIds;
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              taskPromises.push(parsePage(relationValues[k]['id']));
            }
      
            record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1642.json


In [12]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      Tasks: [Array],
      'Task details': [Array],
      'start time': '14:53',
      'end time': '16:03',
      'Roadmap Item': []
    }
  }
}


## 2.2

In [23]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId) {
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
        //   const relationIds = [];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
        //   record[property] = relationIds;
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id']);
              taskPromises.push(parsePage(relationValues[k]['id']));
              record[`Task ${k + 1} name`] = task_details['Name'];
            }
            
            record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=false)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1655.json


In [24]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task 1 name': 'Solarathon: Retrieval pipeline',
      'Task details': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task 1 name': 'Solarathon: Retrieval pipeline',
      'Task details': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      'Task 1 name': 'Solarathon: Retrieval pipeline',
      'Task details': [Array],
      'start time': '14:53',
      'end time': '16:03',
      'Roadmap Item': []
    }
  }
}


## 2.3

In [40]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId) {
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
        const multi_select_rollups = [
            'Project tags', 
        ]
        for (let i = 0; i < multi_select_rollups.length; i++) {
            const rollup = multi_select_rollups[i];
            parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
        }
        const relations = ['Project', 'Parent-task', 'Sub-tasks'];
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
        //   const relationIds = [];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
        //   record[property] = relationIds;
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id']);
              taskPromises.push(parsePage(relationValues[k]['id']));
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                  record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
            }
            
            record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=false)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details


In [26]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Task details': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Task details': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ce

## Iteration 3: get Project details

In [68]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = ['Tasks'];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        for (let i = 0; i < relations.length; i++) {
            const relation = relations[i];
            //  Only parse relation if it is greater than 0
            if (data['properties'][relation]['relation'].length === 0) {
                parsed_data[relation] = null;
            } else {
                parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
            };
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
        //   const relationIds = [];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
        //   record[property] = relationIds;
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
              taskPromises.push(parsePage(relationValues[k]['id'], database='tasks'));
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                  record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
              const projectId = task_details['Project'];
              record[`Project ${k + 1}`] = await parsePage(projectId, 'projects');
            }
            
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

// var parsed_data = parsePage('1be846c5-ddbf-4823-8c5e-ebed5aa1ac31', 'projects')
// console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1745.json


In [69]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Project 1': [Object],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Project 1': [Object],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90

## 3.1 Works for creating a column for task 

In [75]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskPromises = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
              taskPromises.push(parsePage(relationValues[k]['id'], database='tasks'));
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                  record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              record[`Project ${k + 1}`] = project['Name']
            }
            
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=false)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details


In [76]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Project 1': 'Solarathon',
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task 1 Name': 'Solarathon: Retrieval pipeline',
      'Task 1 Project tags': [Array],
      'Task 1 Project': '1be846c5-ddbf-4823-8c5e-ebed5aa1ac31',
      'Task 1 Parent-task': null,
      'Task 1 Sub-tasks': null,
      'Project 1': 'Solarathon',
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d

## 3.2 Update parsing

In [85]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              const attribute_list = [];
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                attribute_list.push(task_details[attributes[attribute]])
                record[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              attribute_dict[`Project ${k + 1}`] = project['Name']
            }
            
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1801.json


In [86]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      Name: [Array],
      'Project tags': [Array],
      Project: [Array],
      'Parent-task': [Array],
      'Sub-tasks': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      Name: [Array],
      'Project tags': [Array],
      Project: [Array],
      'Parent-task': [Array],
      'Sub-tasks': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      Name: [Array],
      'Project tags': [Array],
      Project: [Array],
      'Parent-task': [Array],
      'Sub-tasks': [Arra

## 3.3

In [87]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              const attribute_list = [];
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                attribute_list.push(task_details[attributes[attribute]])
                attribute_dict[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[task_attributes[c]] = attribute_dict[task_attributes[c]];
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              attribute_dict[`Project ${k + 1}`] = project['Name']
            }
            
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)
console.log(parsed_data)

Parsing task details
Promise { <pending> }


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-25_1806.json


## 3.4 Tasks parsed as an array

In [93]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              taskNames.push(task_details['Name']);
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              const attribute_list = [];
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                attribute_list.push(task_details[attributes[attribute]])
                attribute_dict[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              }
              // const task_attributes = Object.keys(attribute_dict)
              // for (let c = 0; c < task_attributes.length; c++) {
              //   record[task_attributes[c]] = attribute_dict[task_attributes[c]];
              // };
              // const projectId = task_details['Project'];
              // const project = await parsePage(projectId, 'projects');
              // attribute_dict[`Project ${k + 1}`] = project['Name']
            }
            record['Tasks'] = taskNames;
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


Parsing task details


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-26_1133.json


In [92]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      Tasks: [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      Tasks: [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      Tasks: [Array],
      'start time': '14:53',
      'end time': '16:03',
      'Roadmap Item': []
    }
  }
}


## 3.5

In [98]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              taskNames.push(task_details['Name']);
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              const attribute_list = [];
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                attribute_list.push(task_details[attributes[attribute]])
                attribute_dict[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              attribute_dict[`Project ${k + 1}`] = project['Name'];
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task: ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
            record['Tasks'] = taskNames; // Add task names
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


Parsing task details


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-26_1147.json


In [97]:
console.log(parsed_data)

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'Task: Project 1': 'Solarathon',
      Tasks: [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'Task: Project 1': 'Solarathon',
      Tasks: [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Fl

## 3.6

In [101]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              taskNames.push(task_details['Name']);
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                const attribute_list = [];
                attribute_list.push(task_details[attributes[attribute]])
                attribute_dict[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              attribute_dict[`Project ${k + 1}`] = project['Name'];
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task: ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
            record['Tasks'] = taskNames; // Add task names
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


Parsing task details


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-26_1152.json


In [100]:
parsed_data

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'Task: Project 1': 'Solarathon',
      Tasks: [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'Task: Project 1': 'Solarathon',
      Tasks: [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Fl

## 3.7

In [102]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  for (let i = 0; i < 3; i++) {
    // for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
          for (let k = 0; k < relationValues.length; k++) {
            // relationIds.push(relationValues[k]['id']);
          }
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            // const taskNames = [];
            const taskProjects = [];
            const taskProjectTags = [];
            // const task
      
            for (let k = 0; k < relationValues.length; k++) {
              // record[`Task ${k + 1}`] = await parsePage(relationValues[k]['id']);
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              // taskNames.push(task_details['Name']);
              
            //   record[`Task ${k + 1} name`] = task_details['Name'];
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                const attribute_list = [];
                attribute_list.push(task_details[attributes[attribute]]);
                attribute_dict[attributes[attribute]] = attribute_list;
                  // record[`Task ${k + 1} ${attributes[attribute]}`] = task_details[attributes[attribute]];
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              // attribute_dict[`Project ${k + 1}`] = project['Name'];
              const project_attributes = Object.keys(project);
              for (let c = 0; c < project_attributes.length; c++) {
                const attribute_list = [];
                attribute_list.push(project[project_attributes[c]]);
                attribute_dict[project_attributes[c]] = attribute_list;
              };
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task: ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
            // record['Tasks'] = taskNames; // Add task names
            // record['Task details'] = await Promise.all(taskPromises);
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


Parsing task details


Parsing task details
Parsing task details
Saved response to ../data/notion_time_tracking_parsed_2023-12-26_1200.json


In [103]:
parsed_data

Promise {
  {
    'a7f579a6-5439-43b9-9e5d-f695889ac88f': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 3.566666666667,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'start time': '20:26',
      'end time': '24:00',
      'Roadmap Item': []
    },
    '3b00d2c2-7b81-4443-834b-84ed72b2ec7d': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.2,
      'Task: Name': [Array],
      'Task: Project tags': [Array],
      'Task: Project': [Array],
      'Task: Parent-task': [Array],
      'Task: Sub-tasks': [Array],
      'start time': '16:03',
      'end time': '17:15',
      'Roadmap Item': []
    },
    'f93a93d3-5fbc-4c9d-a5d1-ceee90b098ba': {
      'Task Tag': [Array],
      Flag: null,
      Notes: null,
      Elapsed: 1.166666666667,
      'Task: Name': [Array],
      'Task: Project tags': [Arra

## 3.7

In [108]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  // for (let i = 0; i < 3; i++) {
  for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
      
          if (property === 'Tasks') {
            console.log('Parsing task details');
            const taskProjects = [];
            const taskProjectTags = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                const attribute_list = [];
                attribute_list.push(task_details[attributes[attribute]]);
                attribute_dict[attributes[attribute]] = attribute_list;
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              
              let project_attributes = Object.keys(project);
              // console.log(project_attributes);
              for (let c = 0; c < project_attributes.length; c++) {
                const attribute_list = [];
                attribute_list.push(project[project_attributes[c]]);
                if (project_attributes[c] === 'Name') {
                  project_attributes[c] = 'Project name';
                };
                attribute_dict[project_attributes[c]] = attribute_list;
              };
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


Parsing task details


Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task details
Parsing task 

## 3.8 Cleaned version

In [ ]:
var {saveResponseJson} = require('../src/fileFunctions');

async function retrievePage(
    pageId, jsonFileName='../data/notion_page', save = false, appendTimestamp = true
  ) {
    const { Client } = require('@notionhq/client');
    const fs = require('fs');
    let notionApiKey = process.env.notion_secret;
    let notion = new Client({ auth: notionApiKey });
      try {
        const response = await notion.pages.retrieve({ page_id: pageId });
        if (save) {
          await saveResponseJson(response, jsonFileName, appendTimestamp);
        }
        return response;
      } catch (error) {
        console.error(error);
        throw error;
      }
  }
  
async function parsePage(pageId, database='tasks') {
    let multi_select_rollups = [];
    let relations = [];
    try {
        const data = await retrievePage(pageId);
        const parsed_data = {};
        if (database === 'tasks') {
            parsed_data['Name'] = data['properties']['Name']['title'][0]['plain_text'];
            multi_select_rollups = [
                'Project tags', 
            ]
            relations = ['Project', 'Parent-task', 'Sub-tasks'];
        } else if (database === 'projects') {
          
          parsed_data['Name'] = data['properties']['Project name']['title'][0]['plain_text'];
          multi_select_rollups = [];
          relations = [];
        } 
        if (multi_select_rollups.length > 0) {
            for (let i = 0; i < multi_select_rollups.length; i++) {
                const rollup = multi_select_rollups[i];
                parsed_data[rollup] = data['properties'][rollup]['rollup']['array'][0]['multi_select'].map(item => item['name']);
            }            
        }
        if (relations.length > 0) {
            for (let i = 0; i < relations.length; i++) {
                const relation = relations[i];
                //  Only parse relation if it is greater than 0
                if (data['properties'][relation]['relation'].length === 0) {
                    parsed_data[relation] = null;
                } else {
                    parsed_data[relation] = data['properties'][relation]['relation'][0]['id'];
                };
            }
        }
        return parsed_data
    } catch (error) {
        console.log(`Error for database ${database}: ${error}`);
    };
}


async function parseTimeTracking(
  data, save = false, jsonFileName='../data/notion_time_tracking_parsed', appendTimestamp = true
) {
  const parsed_data = {};
  const relations_list = ['Tasks'];
  const array_types = ['multi_select', 'relation'];
  let properties = Object.keys(data[0]['properties']);
  const to_ignore = ['Created time', 'Start min', 'summary', 'End min', 'follow up task', 'URL', 'End hr', 'Start hr', 'Name', 'Projects', 'Project tag', 'Project (Rollup)'];
  properties = properties.filter(item => !to_ignore.includes(item));

  // for (let i = 0; i < 3; i++) {
  for (let i = 0; i < data.length; i++) {
    const item = data[i];
    const id = item['id'];
    const record = {};

    for (let j = 0; j < properties.length; j++) {
      const property = properties[j];
      const property_dict = data[i]['properties'][property];
      const property_type = property_dict['type'];

      if (property_type === 'relation') {
        if (property_dict[property_type].length === 0) {
          record[property] = null;
        } else {
          const relationValues = property_dict[property_type];
      
      
          if (property === 'Tasks') {
            const taskProjects = [];
            const taskProjectTags = [];
      
            for (let k = 0; k < relationValues.length; k++) {
              const task_details = await parsePage(relationValues[k]['id'], database='tasks');
              
              const attributes = Object.keys(task_details);
              const attribute_dict = {};
              for (let attribute = 0;attribute < attributes.length; attribute++) {
                const attribute_list = [];
                attribute_list.push(task_details[attributes[attribute]]);
                attribute_dict[attributes[attribute]] = attribute_list;
              };
              const projectId = task_details['Project'];
              const project = await parsePage(projectId, 'projects');
              
              let project_attributes = Object.keys(project);
              // console.log(project_attributes);
              for (let c = 0; c < project_attributes.length; c++) {
                const attribute_list = [];
                attribute_list.push(project[project_attributes[c]]);
                if (project_attributes[c] === 'Name') {
                  project_attributes[c] = 'Project name';
                };
                attribute_dict[project_attributes[c]] = attribute_list;
              };
              const task_attributes = Object.keys(attribute_dict)
              for (let c = 0; c < task_attributes.length; c++) {
                record[`Task ${task_attributes[c]}`] = attribute_dict[task_attributes[c]]; // task attributes
              };
            }
          }
        }
      } else if (property_type === 'rollup') {
        rollup_type = property_dict[property_type]['type'];

        if (rollup_type === 'array' && property_dict[property_type]['array'].length > 0) {
          const array_type = property_dict[property_type]['array'][0]['type'];

          if (array_type === 'multi_select' || array_type === 'relation') {
            record[property] = property_dict[property_type]['array'][0][array_type].map(item => item['name']);
          } else {
            record[property] = null;
          }
        } else {
          record[property] = null;
        }
      } else if (property_type === 'rich_text') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type][0]['text']['content'];
        } else {
          record[property] = null;
        }
      } else if (property_type === 'formula') {
        formula_type = property_dict[property_type]['type'];
        record[property] = property_dict[property_type][formula_type];
      } else if (property_type === 'multi_select') {
        if (property_dict[property_type].length > 0) {
          record[property] = property_dict[property_type].map(item => item);
        } else {
          record[property] = null;
        }
      } else {
        console.log(`Property of type ${property_type} was not parsed: ${property}`);
      }
    }

    parsed_data[id] = record;
  }

  if (save) {
    saveResponseJson(parsed_data, jsonFileName, appendTimestamp);
  }

  return parsed_data;
}

var parsed_data = parseTimeTracking(data, save=true)


# *End of Page*